# Network Models

### Imports

In [2]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization
from keras.layers import RNN, SimpleRNN
from keras.utils import np_utils, plot_model
from keras.callbacks import ModelCheckpoint
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

import numpy as np
from pprint import pprint
import tensorflow as tf
from keras.callbacks import TensorBoard

Using TensorFlow backend.


# ------------------------------------------------

In [5]:
train_x = np.random.rand(100, 3, 2)
train_y = np.random.rand(100, 2)

validation_x = np.random.rand(100, 3, 2)
validation_y = np.random.rand(100, 2)

In [6]:
pprint(train_x.shape[1:])
pprint(train_x[:5])

(3, 2)
array([[[0.32496872, 0.6251876 ],
        [0.6556618 , 0.43552785],
        [0.67985436, 0.37681654]],

       [[0.28830265, 0.34218886],
        [0.44764243, 0.71133462],
        [0.17550684, 0.99066505]],

       [[0.55050903, 0.09398827],
        [0.30096222, 0.57236999],
        [0.57061568, 0.70631687]],

       [[0.28522686, 0.2296998 ],
        [0.72466808, 0.83859367],
        [0.54794638, 0.43529601]],

       [[0.92980911, 0.06783863],
        [0.04827768, 0.5493651 ],
        [0.61037905, 0.45371632]]])


In [7]:
# NAME = f"{SEQ_LEN}-SEQ-{FUTURE_PERIOD_PREDICT}-PRED-{int(time.time())}"

model = Sequential()
model.add(LSTM(
    128,
    input_shape=train_x.shape[1:],
    return_sequences=True
))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
# model.add(Dense(32))
model.add(Dense(2, activation='softmax'))
model.compile(
    loss='categorical_crossentropy',
    optimizer='rmsprop',
    metrics=['accuracy']
)
model.summary()

tensorboard = TensorBoard(log_dir="logs/{}")

filepath = "RNN_Final-{epoch:02d}"  # unique file name that will include the epoch and the validation acc for that epoch
# checkpoint = ModelCheckpoint("models/{}.model".format(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')) # saves only the best ones

checkpoint = ModelCheckpoint(
    filepath,
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min',
)

# Train model
history = model.fit(
    train_x, train_y,
    batch_size=1,
    epochs=3,
#     validation_data=(validation_x, validation_y),
    callbacks=[checkpoint],
)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 3, 128)            67072     
_________________________________________________________________
dropout_1 (Dropout)          (None, 3, 128)            0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 258       
Total params: 198,914
Trainable params: 198,914
Non-trainable params: 0
_________________________________________________________________
Epoch 1/3


KeyboardInterrupt: 

In [5]:
# Score model
score = model.evaluate(validation_x, validation_y, verbose=0)
print('Test loss:', score)
print('Test accuracy:', score)
# Save model
model.save("models/model")

Test loss: [0.6759735202789307, 0.5]
Test accuracy: [0.6759735202789307, 0.5]


In [6]:
pred_input = np.random.rand(1,3,2)

pred = model.predict(pred_input)

pprint(pred.tolist())

[[0.48275941610336304, 0.517240583896637]]


## LSTM -> Dropout -> LSTM -> Dropout -> Dense

In [71]:
model = Sequential()
model.add(LSTM(
    128,
    input_shape=train_x.shape[1:],
    return_sequences=True
))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(2, activation='softmax'))
model.compile(
    loss='categorical_crossentropy',
    optimizer='rmsprop',
    metrics=['accuracy']
)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_41 (LSTM)               (None, 3, 128)            67072     
_________________________________________________________________
dropout_14 (Dropout)         (None, 3, 128)            0         
_________________________________________________________________
lstm_42 (LSTM)               (None, 128)               131584    
_________________________________________________________________
dropout_15 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_44 (Dense)             (None, 2)                 258       
Total params: 198,914
Trainable params: 198,914
Non-trainable params: 0
_________________________________________________________________


## LSTM -> Dense

In [8]:
model = Sequential()
model.add(LSTM(
    128,
    input_shape=train_x.shape[1:])) ## change to (maxlen, len(notes))
model.add(Dense(52, activation='softmax')) ## change to len(notes)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 128)               67072     
_________________________________________________________________
dense_2 (Dense)              (None, 52)                6708      
Total params: 73,780
Trainable params: 73,780
Non-trainable params: 0
_________________________________________________________________


## LSTM -> Dropout -> LSTM -> Dropout -> LSTM -> Dense -> Dropout -> Dense

In [8]:
model = Sequential()
model.add(LSTM(
    32,
    input_shape=(2,),
    return_sequences=True
))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.summary()

ValueError: Input 0 is incompatible with layer lstm_4: expected ndim=3, found ndim=2